In [36]:
import numpy as np
import pandas as pd

import shapely
from shapely import wkt
from shapely.geometry import LineString, MultiLineString, Polygon, Point

In [37]:
# download data
links = pd.read_csv("./data/links.csv")
speed_limit = pd.read_csv("./data/speed_limits.csv")

In [38]:
# change string to shapely geometries
speed_limit['the_geom'] = speed_limit['the_geom'].apply(wkt.loads)

In [39]:
speed_limit_kmh = dict()
speed_limit_kmh_buffered_geom = dict()

for idx, row in speed_limit.iterrows():
    # speed limits in the table have mph units
    mph = int(row['postvz_sl'])
    
    # if there is no limit, set as default (25 mph)
    if mph == 0:
        mph = 25
        
    # mph -> km/h
    speed_limit_kmh[row['the_geom']] = 1.609344 * mph
    # buffered
    speed_limit_kmh_buffered_geom[row['the_geom'].buffer(0.001)] = 1.609344 * mph

In [40]:
buffered_links_in_ny = np.array(list(speed_limit_kmh_buffered_geom.keys()))

In [41]:
time = list()
limit = list()

for idx, row in links.iterrows():
    ego_link = LineString([Point(row['startX'], row['startY']), Point(row['endX'], row['endY'])])
    street_length = float(row['street_length'])
    
    # find geometries containing the link
    related_links = np.array(shapely.contains(buffered_links_in_ny, ego_link))

    if not any(related_links):
        # put 25 mph if there is no corresponding geometry
        t_ = street_length / ((1.609344 * 25) / 3.6)
        l_ = (1.609344 * 25) / 3.6
    else:
        ratio = 0
        corr_link = None
        # put the speed limit of the most overlapping geometry
        for cand in buffered_links_in_ny[related_links]:
            intersects = shapely.intersection(ego_link, cand)
            r_ = intersects.length / ego_link.length
            if r_ > ratio:
                ratio = r_
                corr_link = cand
        t_ = street_length / (speed_limit_kmh_buffered_geom[corr_link] / 3.6)
        l_ = speed_limit_kmh_buffered_geom[corr_link] / 3.6
    # save in list
    time.append(t_)
    limit.append(l_)

In [20]:
# add new columns for travel time attribute
links['travel_time'] = time
links['speed_limit'] = limit

In [43]:
links.to_csv('links_L260855.csv', index=False)

In [44]:
print("maximum speed limit is ", max(speed_limit_kmh_buffered_geom.values()) * 1.609344, "km/h")

maximum speed limit is  129.4994055168 km/h
